In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import plotly.graph_objs as go 
import plotly.offline as py 
import math
from collections import Counter

In [3]:

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import VotingClassifier
from sklearn.base import clone 
import xgboost as xgb

In [5]:
df_wallets_features_classes_combined = pd.read_csv('/kaggle/input/elliptic/wallets_features_classes_combined.csv')

In [6]:
df_wallets_features_classes_combined.columns

Index(['address', 'Time step', 'class', 'num_txs_as_sender',
       'num_txs_as receiver', 'first_block_appeared_in',
       'last_block_appeared_in', 'lifetime_in_blocks', 'total_txs',
       'first_sent_block', 'first_received_block', 'num_timesteps_appeared_in',
       'btc_transacted_total', 'btc_transacted_min', 'btc_transacted_max',
       'btc_transacted_mean', 'btc_transacted_median', 'btc_sent_total',
       'btc_sent_min', 'btc_sent_max', 'btc_sent_mean', 'btc_sent_median',
       'btc_received_total', 'btc_received_min', 'btc_received_max',
       'btc_received_mean', 'btc_received_median', 'fees_total', 'fees_min',
       'fees_max', 'fees_mean', 'fees_median', 'fees_as_share_total',
       'fees_as_share_min', 'fees_as_share_max', 'fees_as_share_mean',
       'fees_as_share_median', 'blocks_btwn_txs_total', 'blocks_btwn_txs_min',
       'blocks_btwn_txs_max', 'blocks_btwn_txs_mean', 'blocks_btwn_txs_median',
       'blocks_btwn_input_txs_total', 'blocks_btwn_input_txs_min'

In [8]:
df_wallets_features_classes_combined

,address,Time step,class,num_txs_as_sender,num_txs_as receiver,first_block_appeared_in,last_block_appeared_in,lifetime_in_blocks,total_txs,first_sent_block,...,blocks_btwn_output_txs_min,blocks_btwn_output_txs_max,blocks_btwn_output_txs_mean,blocks_btwn_output_txs_median,num_addr_transacted_multiple,transacted_w_address_total,transacted_w_address_min,transacted_w_address_max,transacted_w_address_mean,transacted_w_address_median
0,111112TykSw72ztDN2WJger4cynzWYC5w,25,2,0.0,1.0,439586.0,439586.0,0.0,1.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,24.0,1.0,1.0,1.0,1.0
1,1111DAYXhoxZx2tsRnzimfozo783x1yC2,25,3,0.0,8.0,439589.0,485959.0,46370.0,8.0,0.0,...,0.0,20164.0,6624.285714,8060.0,0.0,8.0,1.0,1.0,1.0,1.0
2,1111DAYXhoxZx2tsRnzimfozo783x1yC2,29,3,0.0,8.0,439589.0,485959.0,46370.0,8.0,0.0,...,0.0,20164.0,6624.285714,8060.0,0.0,8.0,1.0,1.0,1.0,1.0
3,1111DAYXhoxZx2tsRnzimfozo783x1yC2,39,3,0.0,8.0,439589.0,485959.0,46370.0,8.0,0.0,...,0.0,20164.0,6624.285714,8060.0,0.0,8.0,1.0,1.0,1.0,1.0
4,1111DAYXhoxZx2tsRnzimfozo783x1yC2,39,3,0.0,8.0,439589.0,485959.0,46370.0,8.0,0.0,...,0.0,20164.0,6624.285714,8060.0,0.0,8.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268255,3R2Uw5MRdSSigp8AjfT7K5es6Hupm4qLSq,31,3,1.0,1.0,451684.0,451685.0,1.0,2.0,451685.0,...,0.0,0.0,0.000000,0.0,0.0,3.0,1.0,1.0,1.0,1.0
1268256,3R2VBFbqHGC4bQ7b4ixN4jZTdv7RMbEYtf,44,3,0.0,1.0,477895.0,477895.0,0.0,1.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,1.0,1.0,1.0,1.0,1.0
1268257,3R2WFmRwbDeo3rMVVu5J3jjMxAuQYYWAid,9,3,0.0,1.0,407342.0,407342.0,0.0,1.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,1.0,1.0,1.0,1.0,1.0
1268258,3R2WTZGYLmbJQyoDSBftJsPRvF1mSEtkh6,3,3,0.0,1.0,395235.0,395235.0,0.0,1.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,1.0,1.0,1.0,1.0,1.0


In [7]:
#Lọc bỏ timestep và các rows trùng nhau
df_wallets_classification = df_wallets_features_classes_combined
df_wallets_classification = df_wallets_classification.drop(columns=['Time step']).drop_duplicates()
df_wallets_classification

,address,class,num_txs_as_sender,num_txs_as receiver,first_block_appeared_in,last_block_appeared_in,lifetime_in_blocks,total_txs,first_sent_block,first_received_block,...,blocks_btwn_output_txs_min,blocks_btwn_output_txs_max,blocks_btwn_output_txs_mean,blocks_btwn_output_txs_median,num_addr_transacted_multiple,transacted_w_address_total,transacted_w_address_min,transacted_w_address_max,transacted_w_address_mean,transacted_w_address_median
0,111112TykSw72ztDN2WJger4cynzWYC5w,2,0.0,1.0,439586.0,439586.0,0.0,1.0,0.0,439586.0,...,0.0,0.0,0.000000,0.0,0.0,24.0,1.0,1.0,1.0,1.0
1,1111DAYXhoxZx2tsRnzimfozo783x1yC2,3,0.0,8.0,439589.0,485959.0,46370.0,8.0,0.0,439589.0,...,0.0,20164.0,6624.285714,8060.0,0.0,8.0,1.0,1.0,1.0,1.0
9,1111VHuXEzHaRCgXbVwojtaP7Co3QABb,2,0.0,1.0,431522.0,431522.0,0.0,1.0,0.0,431522.0,...,0.0,0.0,0.000000,0.0,0.0,1.0,1.0,1.0,1.0,1.0
10,111218KKkh1JJFRHbwM16AwCiVCc4m7he1,3,1.0,1.0,423456.0,423456.0,0.0,2.0,423456.0,423456.0,...,0.0,0.0,0.000000,0.0,0.0,4.0,1.0,1.0,1.0,1.0
12,1115LWW3xsD9jT9VRY7viCN9S34RVAAuA,2,0.0,1.0,429513.0,429513.0,0.0,1.0,0.0,429513.0,...,0.0,0.0,0.000000,0.0,0.0,5.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268254,3R2Uw5MRdSSigp8AjfT7K5es6Hupm4qLSq,3,1.0,1.0,451684.0,451685.0,1.0,2.0,451685.0,451684.0,...,0.0,0.0,0.000000,0.0,0.0,3.0,1.0,1.0,1.0,1.0
1268256,3R2VBFbqHGC4bQ7b4ixN4jZTdv7RMbEYtf,3,0.0,1.0,477895.0,477895.0,0.0,1.0,0.0,477895.0,...,0.0,0.0,0.000000,0.0,0.0,1.0,1.0,1.0,1.0,1.0
1268257,3R2WFmRwbDeo3rMVVu5J3jjMxAuQYYWAid,3,0.0,1.0,407342.0,407342.0,0.0,1.0,0.0,407342.0,...,0.0,0.0,0.000000,0.0,0.0,1.0,1.0,1.0,1.0,1.0
1268258,3R2WTZGYLmbJQyoDSBftJsPRvF1mSEtkh6,3,0.0,1.0,395235.0,395235.0,0.0,1.0,0.0,395235.0,...,0.0,0.0,0.000000,0.0,0.0,1.0,1.0,1.0,1.0,1.0


In [11]:
# Bỏ unknown (class == 3) - we're classifying only 2 class (1 & 2)
data = df_wallets_classification.loc[(df_wallets_classification['class'] != 3), 'address']
df_wallets_feature_selected = df_wallets_classification.loc[df_wallets_classification['address'].isin(data)]
df_wallets_feature_selected
# Cuz the actor 

,address,class,num_txs_as_sender,num_txs_as receiver,first_block_appeared_in,last_block_appeared_in,lifetime_in_blocks,total_txs,first_sent_block,first_received_block,...,blocks_btwn_output_txs_min,blocks_btwn_output_txs_max,blocks_btwn_output_txs_mean,blocks_btwn_output_txs_median,num_addr_transacted_multiple,transacted_w_address_total,transacted_w_address_min,transacted_w_address_max,transacted_w_address_mean,transacted_w_address_median
0,111112TykSw72ztDN2WJger4cynzWYC5w,2,0.0,1.0,439586.0,439586.0,0.0,1.0,0.0,439586.0,...,0.0,0.0,0.0,0.0,0.0,24.0,1.0,1.0,1.0,1.0
9,1111VHuXEzHaRCgXbVwojtaP7Co3QABb,2,0.0,1.0,431522.0,431522.0,0.0,1.0,0.0,431522.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
12,1115LWW3xsD9jT9VRY7viCN9S34RVAAuA,2,0.0,1.0,429513.0,429513.0,0.0,1.0,0.0,429513.0,...,0.0,0.0,0.0,0.0,0.0,5.0,1.0,1.0,1.0,1.0
13,1117wASFaYgJJP6MiY8cPD5DMdQda8gDZ,2,1.0,1.0,399269.0,399271.0,2.0,2.0,399271.0,399269.0,...,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,1.0,1.0
15,111ECNFEjJsQJ6LkAsAESBGTHBvZE66oX,2,0.0,1.0,423462.0,423462.0,0.0,1.0,0.0,423462.0,...,0.0,0.0,0.0,0.0,0.0,11.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268243,3R2KQxz2Uk7xF9bfDGffTn52NrfnBjdaLy,2,0.0,1.0,411364.0,411364.0,0.0,1.0,0.0,411364.0,...,0.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,1.0,1.0
1268244,3R2LAgPtnjeU2XdyR1GUXaTzKmh81VETYm,2,0.0,1.0,435555.0,435555.0,0.0,1.0,0.0,435555.0,...,0.0,0.0,0.0,0.0,0.0,5.0,1.0,1.0,1.0,1.0
1268251,3R2SZEK2UggNm1ntkXTe8eoea11sxg9vjw,2,0.0,1.0,421443.0,421443.0,0.0,1.0,0.0,421443.0,...,0.0,0.0,0.0,0.0,0.0,9.0,1.0,1.0,1.0,1.0
1268252,3R2TfX1PDN93SHbDXZqYPCn1cgmCeLMBra,2,0.0,1.0,437572.0,437572.0,0.0,1.0,0.0,437572.0,...,0.0,0.0,0.0,0.0,0.0,11.0,1.0,1.0,1.0,1.0


In [30]:
df_wallets_feature_selected.info()

<class 'pandas.core.frame.DataFrame'>
Index: 265354 entries, 0 to 1268253
Data columns (total 57 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   address                        265354 non-null  object 
 1   class                          265354 non-null  int64  
 2   num_txs_as_sender              265354 non-null  float64
 3   num_txs_as receiver            265354 non-null  float64
 4   first_block_appeared_in        265354 non-null  float64
 5   last_block_appeared_in         265354 non-null  float64
 6   lifetime_in_blocks             265354 non-null  float64
 7   total_txs                      265354 non-null  float64
 8   first_sent_block               265354 non-null  float64
 9   first_received_block           265354 non-null  float64
 10  num_timesteps_appeared_in      265354 non-null  float64
 11  btc_transacted_total           265354 non-null  float64
 12  btc_transacted_min             265

In [12]:
# Goal: binary classification of 0,1
# 0: licit, 1: illicit

# change illicit (class-2) to '0' for classification
y = df_wallets_feature_selected[['class']]
y = y['class'].apply(lambda x: 0 if x == 2 else 1 ) 

X_train, X_test, y_train, y_test = train_test_split(df_wallets_feature_selected,y,test_size=0.30,random_state=15, shuffle=False)
X_train = X_train.drop(columns=['address', 'class'])
X_test = X_test.drop(columns=['address', 'class'])

In [13]:
y

0          0
9          0
12         0
13         0
15         0
          ..
1268243    0
1268244    0
1268251    0
1268252    0
1268253    0
Name: class, Length: 265354, dtype: int64

In [14]:
# Scaling
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

#Flatten into 1 dimension array
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

# Test on different models

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_recall_fscore_support, f1_score

In [1]:
def get_model_metrics(y_true, y_pred):
    prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, zero_division=0)
    micro_f1 = f1_score(y_true, y_pred, average='micro')
    accuracy = accuracy_score(y_true, y_pred)
    return {
        'Precision': prec[1],
        'Recall': rec[1],
        'F1 Score': f1[1],
        'Micro F1': micro_f1,
        'Accuracy': accuracy
    }

In [ ]:
results = []

In [25]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)
metrics_lr = get_model_metrics(y_test, y_pred_lr)
metrics_lr['Model'] = 'Logistic Regression'
results.append(metrics_lr)


📌 Logistic Regression
Precision: 0.524
Recall: 0.060
F1 Score: 0.108
Micro-Average F1: 0.964


In [26]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)
metrics_rf = get_model_metrics(y_test, y_pred_rf)
metrics_rf['Model'] = 'Random Forest'
results.append(metrics_rf)


📌 Random Forest
Precision: 0.921
Recall: 0.790
F1 Score: 0.850
Micro-Average F1: 0.990


In [29]:
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X_train_scaled, y_train)
y_pred_xgb = xgb.predict(X_test_scaled)
metrics_xgb = get_model_metrics(y_test, y_pred_xgb)
metrics_xgb['Model']='XGB'
results.append(metrics_xgb)


 📌 XGBoost
Precision: 0.888
Recall: 0.809
F1 Score: 0.846
Micro-Average F1: 0.989


In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
mlp.fit(X_train_scaled, y_train)
y_pred_mlp = mlp.predict(X_test_scaled)
metrics_mlp = get_model_metrics(y_test, y_pred_mlp)
metrics_mlp['Model'] = 'Multi-Layer Perceptron'
results.append(metrics_mlp)

In [ ]:
lgbm = LGBMClassifier(random_state=42, n_estimators=100)
lgbm.fit(X_train_scaled, y_train)
y_pred_lgbm = lgbm.predict(X_test_scaled)
metrics_lgbm = get_model_metrics(y_test, y_pred_lgbm)
metrics_lgbm['Model']='LGBM'
results.append(metrics_lgbm)

# Run Ensemble Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import precision_recall_fscore_support, f1_score

In [ ]:
ensembles = {
    "RF + MLP": [("rf", rf), ("mlp", mlp)],
    "RF + XGB": [("rf", rf), ("xgb", xgb)],
    "MLP + XGB": [("mlp", mlp), ("xgb", xgb)],
    "RF + MLP + XGB": [("rf", rf), ("mlp", mlp), ("xgb", xgb)],
}

In [ ]:
for name, estimators in ensembles.items():
    clf = VotingClassifier(estimators=estimators, voting='hard')
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)
    metrics_ensemble = get_model_metrics(y_test, y_preds)
    metrics_ensemble['Model']= name
    results.append(metrics_ensemble)

In [ ]:
df_results = pd.DataFrame(results)
df_results = df_results[['Model', 'Precision', 'Recall', 'F1 Score', 'Micro F1', 'Accuracy']]  # order columns

print(df_results)
